In [83]:
%load_ext autoreload
%autoreload 2

import sys, os
from pathlib import Path
import plotly.graph_objects as go

# Fix import
sys.path.append(os.path.abspath(os.path.join('..')))
from facilitators import HourlyTechActivityRateSolutionCompareFacilitator, StackedQuantityEvolutionFacilitatorBase
from data import HEADER_MAPPING

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [84]:
PROJECT_ROOT_DIR = Path().resolve().parent
SOLUTION_DIR = PROJECT_ROOT_DIR / "solution_files"

DAILY_DATA = SOLUTION_DIR / "solution_data_daily_d20_h6_s6_no_invest.sol"
SEASON_DATA = SOLUTION_DIR / "solution_data_season_d20_h6_s6_no_invest.sol"

REGION = "ALL"

# Capacity Evolution

In [95]:

def storage_capacity_evolution_fig(sol_path: str, region: str = "ALL", title_addon: str = None) -> go.Figure:
    storage_evolution = StackedQuantityEvolutionFacilitatorBase(sol_path=sol_path, type_of_data_to_read="TotalCapacityAnnual", 
                                                                region=region, extra_identifying_columns=["Technology"])

    d = storage_evolution.get_relevant_data()
    d.filter_by_containing_string(column="Technology", identifier="D_")

    if region == "ALL":
        d.aggreagate_all_by_sum(column_to_aggregate="Region", aggregated_entry_name=region, column_to_sum="Value")

    fig = storage_evolution.plot(storage_evolution.generate_traces(d))

    fig.update_layout(
        width=1400,
        height=800,
        title=f"Storage Capacity, Region: {region}" + f", {title_addon}" if title_addon is not None else "",
        yaxis_title="GW",
        xaxis_title="Year",
    )

    return fig

"""storage_evolution = StackedQuantityEvolutionFacilitatorBase(sol_path=SEASON_DATA, type_of_data_to_read="TotalCapacityAnnual", 
                                                            region=REGION, extra_identifying_columns=["Technology"])

d = storage_evolution.get_relevant_data()
d.filter_by_containing_string(column="Technology", identifier="D_")

if REGION == "ALL":
    d.aggreagate_all_by_sum(column_to_aggregate="Region", aggregated_entry_name=REGION, column_to_sum="Value")

fig = storage_evolution.plot(storage_evolution.generate_traces(d))

fig.update_layout(
    width=1400,
    height=800,
    title=f"Storage Capacity, Region: {REGION}, ",
    yaxis_title="GW",
    xaxis_title="Year",
)"""




'storage_evolution = StackedQuantityEvolutionFacilitatorBase(sol_path=SEASON_DATA, type_of_data_to_read="TotalCapacityAnnual", \n                                                            region=REGION, extra_identifying_columns=["Technology"])\n\nd = storage_evolution.get_relevant_data()\nd.filter_by_containing_string(column="Technology", identifier="D_")\n\nif REGION == "ALL":\n    d.aggreagate_all_by_sum(column_to_aggregate="Region", aggregated_entry_name=REGION, column_to_sum="Value")\n\nfig = storage_evolution.plot(storage_evolution.generate_traces(d))\n\nfig.update_layout(\n    width=1400,\n    height=800,\n    title=f"Storage Capacity, Region: {REGION}, ",\n    yaxis_title="GW",\n    xaxis_title="Year",\n)'

In [96]:
storage_capacity_evolution_fig(DAILY_DATA, region=REGION, title_addon="Daily").show()
storage_capacity_evolution_fig(SEASON_DATA, region=REGION, title_addon="Seasonal").show()

No unit conversion applied!


No unit conversion applied!


# Storage net charge

In [97]:
def storage_rate_of_activity_fig(tech: str, region: str = "ALL") -> go.Figure:

    hourly_comparer = HourlyTechActivityRateSolutionCompareFacilitator(tech = tech, sol_paths = {"daily": DAILY_DATA, "season": SEASON_DATA},
                                                                        year = 2050, region = region)

    # Get data
    d = hourly_comparer.get_relevant_data()

    # Keep only modes 1 and 2
    d.filter_by_list("Mode", [1, 2])

    # Set mode 1 to negative
    d.df.loc[d.df["Mode"] == 2, "Value"] *= -1

    # Aggregate over Mode
    d.aggreagate_all_by_sum(column_to_aggregate="Mode", aggregated_entry_name=-1, column_to_sum="Value")

    if region == "ALL":
        d.aggreagate_all_by_sum(column_to_aggregate="Region", aggregated_entry_name=region, column_to_sum="Value")

    fig = hourly_comparer.plot(hourly_comparer.generate_traces(d))
    fig.update_layout(title = f"Storage Net Charge/Discharge, Tech: {tech}, Region: {region}",
                      )

    return fig

In [101]:
storage_rate_of_activity_fig(tech="D_Heat_HLB", region=REGION).show()
storage_rate_of_activity_fig(tech="D_Battery_Li-Ion", region=REGION).show()
storage_rate_of_activity_fig(tech="D_Gas_H2", region=REGION).show()

Converting from PetaJoules to TerraWattHours
Converting from PetaJoules to TerraWattHours


Converting from PetaJoules to TerraWattHours
Converting from PetaJoules to TerraWattHours


Converting from PetaJoules to TerraWattHours
Converting from PetaJoules to TerraWattHours
